In [ ]:
!wget http://www.cs.cmu.edu/~ark/personas/data/MovieSummaries.tar.gz
!tar -xf MovieSummaries.tar.gz

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import nltk
import torch

from transformers import AutoTokenizer, AutoModel
from operator import itemgetter
from torch.nn.functional import normalize

nltk.download('punkt')

In [4]:
# read as dataframe
df_english = pd.read_csv('/content/MovieSummaries/plot_summaries.txt', delimiter = '\t',names=['id','text'])
df_english.info()
df_english = df_english.head(3)

cos = torch.nn.CosineSimilarity(dim = 1)
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

docs_vectors_dics_list = []
for i in df_english.index:
  doc = df_english['text'][i]
  doc_sents = nltk.sent_tokenize(doc)
  for sent in doc_sents:
    inputs = tokenizer(sent, return_tensors='pt')
    outputs = model(**inputs)
    docs_vectors_dics_list.append({
        'doc-index':i,
        'vector':outputs.pooler_output,
        'similarity':0
        })

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42303 entries, 0 to 42302
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      42303 non-null  int64 
 1   text    42303 non-null  object
dtypes: int64(1), object(1)
memory usage: 661.1+ KB


In [ ]:
while(True):
  query = input('Enter your query: ')
  inputs = tokenizer(query, return_tensors='pt')
  outputs = model(**inputs)
  query_vector = outputs.pooler_output
  normal_query_vector = normalize(outputs.pooler_output,2,0)

  for dic in docs_vectors_dics_list:
    dic['similarity'] = cos(dic['vector'],normal_query_vector)

  newlist = sorted(docs_vectors_dics_list, key=itemgetter('similarity'), reverse=True)

  counter = 0
  last_index = -1

  for dic in newlist:
    if(last_index != dic['doc-index'] & counter < 5):
      last_index = dic['doc-index']
      print(last_index)
      counter = counter + 1

  for dic in docs_vectors_dics_list:
    dic['similarity'] = 0

Enter your query: go
1
2
1
2
2
1
1
2
1
2
1
2
2
1
1
2
1
2
1
2
1
2
1
2
1
2
0
2
2
1
1
2
1
2
1
2
1
2
